<a href="https://colab.research.google.com/github/sanket-choudhary-12/IMDB-reviews---Sentiment-Analysis---LSTM/blob/main/IMDB_reviews_Sentiment_Analysis_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import os
import json

import pandas as pd
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [30]:
kaggle_dictionary=json.load(open('/content/kaggle (4).json'))

In [31]:
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [32]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [33]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip  'kaggle (4).json'   sample_data


In [34]:
with ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip','r')as zip_ref:
  zip_ref.extractall()

In [35]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip  'kaggle (4).json'   sample_data


In [36]:
df=pd.read_csv('/content/IMDB Dataset.csv')

In [37]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [38]:
df.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [39]:
df.shape

(50000, 2)

In [40]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [41]:
import numpy as np
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)


In [42]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [43]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [44]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [46]:
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
X_train=pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen=200)
X_test=pad_sequences(tokenizer.texts_to_sequences(test_data['review']),maxlen=200)


In [47]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [48]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [49]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [50]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [51]:

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [52]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [54]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 [==============================] - 231s 451ms/step - loss: 0.4013 - accuracy: 0.8182 - val_loss: 0.3177 - val_accuracy: 0.8626
Epoch 2/5
500/500 [==============================] - 208s 417ms/step - loss: 0.2722 - accuracy: 0.8910 - val_loss: 0.3212 - val_accuracy: 0.8660
Epoch 3/5
500/500 [==============================] - 199s 397ms/step - loss: 0.2222 - accuracy: 0.9139 - val_loss: 0.3138 - val_accuracy: 0.8749
Epoch 4/5
500/500 [==============================] - 199s 397ms/step - loss: 0.1969 - accuracy: 0.9225 - val_loss: 0.3215 - val_accuracy: 0.8716
Epoch 5/5
500/500 [==============================] - 200s 401ms/step - loss: 0.1727 - accuracy: 0.9354 - val_loss: 0.3844 - val_accuracy: 0.8748


In [55]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 [==============================] - 20s 62ms/step - loss: 0.3605 - accuracy: 0.8797
Test Loss: 0.3604671061038971
Test Accuracy: 0.8797000050544739


In [56]:
def predict_sentiment(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [57]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 268ms/step
The sentiment of the review is: positive


In [58]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 79ms/step
The sentiment of the review is: negative
